In [1]:
# Load libraries
import numpy as np
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd

# Set random seed
np.random.seed(0)

Using TensorFlow backend.


In [41]:
# Importing the datasets
dataset= pd.read_csv(r'C:\Users\yosep\Desktop\ANN Keras training\Database Final 3.csv', nrows=95)
data_categories=dataset[['Duration (years)','Road Category','Type of Network Intervention','Number of Province','Geographical Profile','EHS/GHS','GDP','3 years Average GDP Growth','RDP','Change of Power','Ideoogy ','Population','c/km2']]
data_encoded=pd.get_dummies(data_categories)
print(data_encoded.columns)

data_encoded.head()

Index(['Duration (years)', 'Number of Province', 'GDP',
       '3 years Average GDP Growth', 'RDP', 'Population', 'c/km2',
       'Road Category_National', 'Road Category_Provincial',
       'Type of Network Intervention_Modification',
       'Type of Network Intervention_New', 'Geographical Profile_Combination',
       'Geographical Profile_Rural', 'Geographical Profile_Urban',
       'EHS/GHS_No', 'EHS/GHS_Yes', 'Change of Power_No',
       'Change of Power_Yes', 'Ideoogy _Center', 'Ideoogy _Center-Left',
       'Ideoogy _Center-Right ', 'Ideoogy _CenterLeft'],
      dtype='object')


,Duration (years),Number of Province,GDP,3 years Average GDP Growth,RDP,Population,c/km2,Road Category_National,Road Category_Provincial,Type of Network Intervention_Modification,...,Geographical Profile_Rural,Geographical Profile_Urban,EHS/GHS_No,EHS/GHS_Yes,Change of Power_No,Change of Power_Yes,Ideoogy _Center,Ideoogy _Center-Left,Ideoogy _Center-Right,Ideoogy _CenterLeft
0,6.67,2,414838,0.053704,0.2715,3601880,20357.332510,1,0,1,...,1,0,0,1,1,0,0,0,0,1
1,5.04,1,545609,0.033351,0.0612,1136695,6868.199366,1,0,1,...,1,0,1,0,1,0,1,0,0,0
2,2.95,1,523939,0.032014,0.1864,2587265,11389.300570,1,0,0,...,0,0,0,1,1,0,1,0,0,0
3,4.38,1,364961,0.057198,0.0826,1079428,9363.410326,1,0,1,...,1,0,0,1,0,1,0,0,0,1
4,2.42,1,639163,0.054203,0.0844,1201350,10836.177000,1,0,1,...,0,0,0,1,0,1,0,0,1,0


In [42]:
#Identify input features and output (X and y)
X = data_encoded.drop(['Duration (years)'], axis=1)
y = data_encoded.iloc[:, 0].values
y = np.array(y).reshape((-1, 1))

from sklearn.preprocessing import StandardScaler
sc = StandardScaler(1)
features= sc.fit_transform(X)
output=sc.fit_transform(y)

C:\Users\yosep\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\yosep\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [51]:
# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=11, activation='relu', input_shape=(21,)))
    
    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=11, activation='relu'))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='linear'))

    # Compile neural network
    network.compile(loss='mean_squared_error', # Regression Problem
                    optimizer='adam', # Adam
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [52]:
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=100, 
                                 batch_size=9, 
                                 verbose=2)

In [53]:
# Evaluate neural network using three-fold cross-validation
cross_val_score(neural_network, features, output, cv=3)

Epoch 1/100
 - 1s - loss: 1030.9555 - acc: 0.0000e+00
Epoch 2/100
 - 0s - loss: 1021.2814 - acc: 0.0159
Epoch 3/100
 - 0s - loss: 1012.3087 - acc: 0.0159
Epoch 4/100
 - 0s - loss: 1004.1545 - acc: 0.0159
Epoch 5/100
 - 0s - loss: 994.8125 - acc: 0.0159
Epoch 6/100
 - 0s - loss: 985.9871 - acc: 0.0159
Epoch 7/100
 - 0s - loss: 977.0958 - acc: 0.0159
Epoch 8/100
 - 0s - loss: 968.0270 - acc: 0.0317
Epoch 9/100
 - 0s - loss: 958.1239 - acc: 0.0317
Epoch 10/100
 - 0s - loss: 948.2584 - acc: 0.0317
Epoch 11/100
 - 0s - loss: 938.3031 - acc: 0.0317
Epoch 12/100
 - 0s - loss: 927.3675 - acc: 0.0159
Epoch 13/100
 - 0s - loss: 916.0284 - acc: 0.0159
Epoch 14/100
 - 0s - loss: 904.5275 - acc: 0.0159
Epoch 15/100
 - 0s - loss: 892.1053 - acc: 0.0000e+00
Epoch 16/100
 - 0s - loss: 879.4406 - acc: 0.0000e+00
Epoch 17/100
 - 0s - loss: 866.1525 - acc: 0.0159
Epoch 18/100
 - 0s - loss: 852.3178 - acc: 0.0159
Epoch 19/100
 - 0s - loss: 837.6891 - acc: 0.0159
Epoch 20/100
 - 0s - loss: 822.4189 - acc: 

Epoch 63/100
 - 0s - loss: 180.0577 - acc: 0.0159
Epoch 64/100
 - 0s - loss: 177.4383 - acc: 0.0159
Epoch 65/100
 - 0s - loss: 174.7076 - acc: 0.0159
Epoch 66/100
 - 0s - loss: 172.5125 - acc: 0.0159
Epoch 67/100
 - 0s - loss: 170.6428 - acc: 0.0317
Epoch 68/100
 - 0s - loss: 168.7156 - acc: 0.0476
Epoch 69/100
 - 0s - loss: 166.7318 - acc: 0.0476
Epoch 70/100
 - 0s - loss: 165.2498 - acc: 0.0476
Epoch 71/100
 - 0s - loss: 163.6597 - acc: 0.0317
Epoch 72/100
 - 0s - loss: 162.5673 - acc: 0.0317
Epoch 73/100
 - 0s - loss: 160.9788 - acc: 0.0317
Epoch 74/100
 - 0s - loss: 159.4772 - acc: 0.0476
Epoch 75/100
 - 0s - loss: 158.4154 - acc: 0.0317
Epoch 76/100
 - 0s - loss: 157.2847 - acc: 0.0317
Epoch 77/100
 - 0s - loss: 156.2356 - acc: 0.0317
Epoch 78/100
 - 0s - loss: 155.1622 - acc: 0.0317
Epoch 79/100
 - 0s - loss: 154.0702 - acc: 0.0476
Epoch 80/100
 - 0s - loss: 153.2786 - acc: 0.0476
Epoch 81/100
 - 0s - loss: 152.3526 - acc: 0.0476
Epoch 82/100
 - 0s - loss: 151.5437 - acc: 0.0476


array([0.        , 0.03125   , 0.03225806])